In [ ]:
# ------- Scraper ------

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

cache_autores = {}
def obtener_autor(titulo):

    if titulo in cache_autores:
        return cache_autores[titulo]
    
    url_api = "https://openlibrary.org/search.json"
    params = {'title' : titulo}

    try:
        respuesta = requests.get(url_api, params=params, timeout=5)
        if respuesta.status_code == 200:
            data = respuesta.json()
            if data.get("numFound",0) > 0:
                autor = data["docs"][0].get("author_name", ["desconocido"])[0]
            else:
                autor = "desconocido"
        else:
            autor = 'error de API'
    except Exception as e:
        autor = "Error"
    
    cache_autores[titulo] = autor
    time.sleep(1)
    return autor


url = 'https://books.toscrape.com/'
respuesta = requests.get(url)

soup = BeautifulSoup(respuesta.content, 'lxml')

etiqueta_categorias = soup.find('ul', class_='nav nav-list')

# ----- diccionario para guardar links segun categoria -----
category = {}

# ----- para todos los libros -----
link_all_books = url + etiqueta_categorias.a['href']
category['all books'] = link_all_books

# ----- por categorias -----
etiqueta_book_category = etiqueta_categorias.find('ul')
book_category = etiqueta_book_category.find_all('li')
for cat in book_category:
    nombre = cat.a.text.strip()
    link = cat.a['href']
    category[nombre] = url + link 

print("\n-----Categorias-----\n")
for x in category:
    print(x)
category_eleccion = input('categoria a visitar: ')

if category_eleccion in category:

    new_url = category[category_eleccion]
    new_response = requests.get(new_url)
    
    books_category = []
    book_counter = 0

    while True:
        soup_new = BeautifulSoup(new_response.content, 'lxml')

        seleccion_of_books = soup_new.find('ol', class_='row')
        all_books_seleccion = seleccion_of_books.find_all('li')

        for book in all_books_seleccion:

            book_information = book.find('article', class_='product_pod')

            # ---- nombre del libro -----
            title = book_information.h3.a['title']

            # ---- autor del libro -----
            autor = obtener_autor(title)

            # ---- precio del libro -----
            price_tag = book_information.find('div', class_='product_price')
            price = price_tag.find('p', class_='price_color').text

            # ---- disponibilidad del libro ----
            disponibility_tag = price_tag.find('p', class_='instock availability')
            disponibility = disponibility_tag.text.strip()

            # ----- cantidad de estrellas -----
            stars_tag = book_information.find('p', class_='star-rating')
            stars = stars_tag['class'][1] if stars_tag else 'not reviewd'

            # ----- descripcion -----
            description_place_tag_link = book_information.h3.a['href']
            description_place_tag_complete_link = urljoin(new_url, description_place_tag_link)
            description_response = requests.get(description_place_tag_complete_link)
            soup_description = BeautifulSoup(description_response.content, 'lxml')
            product_tag = soup_description.find('div', id='product_description')
            if product_tag:
                des_tag = product_tag.find_next_sibling('p')
                if des_tag:
                    description = des_tag.text.replace('\n', ' ').strip()
            else:
                description = 'No description'

            books_category.append({
                'title' : title,
                'author' : autor,
                'stars' : stars,
                'price' : price,
                'stock' : disponibility,
                'description' : description
            })
            
            book_counter += 1
            print(book_counter, ': new Books')
            


        # ----- si hay mas de una pagina en el apartado -----
        next_tag = soup_new.find('li', class_='next')


        # ----- si hay volver a hacerel requests y trabajar con el siguiente HTML -----
        if next_tag:
            try:
                next_link = next_tag.a['href']
                new_url = urljoin(new_url, next_link)
                new_response = requests.get(new_url)
            except Exception as e:
                print("---- No more to analyse ----")
                break
        else:
            break
    
# terminando esto todo eta en la lista Books -----> primera parte del scraper lista




-----Categorias-----

all books
Travel
Mystery
Historical Fiction
Sequential Art
Classics
Philosophy
Romance
Womens Fiction
Fiction
Childrens
Religion
Nonfiction
Music
Default
Science Fiction
Sports and Games
Add a comment
Fantasy
New Adult
Young Adult
Science
Poetry
Paranormal
Art
Psychology
Autobiography
Parenting
Adult Fiction
Humor
Horror
History
Food and Drink
Christian Fiction
Business
Biography
Thriller
Contemporary
Spirituality
Academic
Self Help
Historical
Christian
Suspense
Short Stories
Novels
Health
Politics
Cultural
Erotica
Crime
1 : new Books
2 : new Books
3 : new Books
4 : new Books
5 : new Books
6 : new Books
7 : new Books
8 : new Books
9 : new Books
10 : new Books
11 : new Books
12 : new Books
13 : new Books
14 : new Books
15 : new Books
16 : new Books
17 : new Books
18 : new Books
19 : new Books
20 : new Books
21 : new Books
22 : new Books
23 : new Books
24 : new Books
25 : new Books
26 : new Books
27 : new Books
28 : new Books
29 : new Books
30 : new Books
31 : new 

ConnectionError: HTTPSConnectionPool(host='books.toscrape.com', port=443): Max retries exceeded with url: /catalogue/mothering-sunday_600/index.html (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000018D5B1E4740>: Failed to resolve 'books.toscrape.com' ([Errno 11001] getaddrinfo failed)"))

In [1]:
# ---- base de datos MySql ----
import sqlite3

# ----- iniciar conexion con base de datos -----
conexion = sqlite3.connect('books_data.db')
cursor = conexion.cursor()

# ---- Crear Tablas -----

cursor.execute("""
CREATE TABLE IF NOT EXISTS Categoria (
    id_category INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
);
 """)

cursor.execute("""
CREATE TABLE IF NOT EXISTS Autor (
    id_author INTEGER PRIMARY KEY AUTOINCREMENT,
    author TEXT UNIQUE
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Libro (
    id_book INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    price REAL,
    stars INTEGER,
    stock TEXT,
    description TEXT,
    id_category INTEGER,
    FOREIGN KEY (id_category) REFERENCES Categoria(id_category)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS LibroAutor (
    id_book INTEGER,
    id_author INTEGER,
    PRIMARY KEY (id_book, id_author),
    FOREIGN KEY (id_book) REFERENCES Libro(id_book),
    FOREIGN KEY (id_author) REFERENCES Autor(id_author)
);
""")

conexion.commit()
conexion.close()

print("base de datos creada")


base de datos creada


In [2]:
import sqlite3

# ----- iniciar conexion -----
conexion = sqlite3.connect('books_data.db')
cursor = conexion.cursor()

category_list_books = category_eleccion.strip().lower()

cursor.execute("INSERT OR IGNORE INTO Categoria (name) VALUES (?)", (category_eleccion,))
conexion.commit()

cursor.execute("SELECT id_category FROM Categoria WHERE name = ?", (category_eleccion,))
id_category = cursor.fetchone()[0]

for book in books_category:
    titulo = book['title']
    autor = book['author']
    estrellas = 0
    try: 
        estrellas = ["Zero","One","Two","Three","Four","Five"].index(book['stars'].capitalize())
    except ValueError:
        estrellas = None

    precio = float(book['price'].replace('£','').strip()) if "£" in book['price'] else None
    stock = book['stock']
    descripcion = book['description']

    cursor.execute("INSERT OR IGNORE INTO Autor (author) VALUES (?)", (autor,))
    conexion.commit()

    cursor.execute("SELECT id_author FROM Autor WHERE author = ?", (autor,))
    id_author = cursor.fetchone()[0]

    cursor.execute("""
        INSERT INTO Libro (title, price, stars, stock, description, id_category)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (titulo, precio, estrellas, stock, descripcion, id_category))
    conexion.commit()

    id_libro = cursor.lastrowid

    cursor.execute("""
        INSERT OR IGNORE INTO LibroAutor (id_book, id_author)
        VALUES (?, ?)
    """, (id_libro, id_author))

conexion.commit()
conexion.close()

print(" Datos insertados correctamente en la base de datos.")

 Datos insertados correctamente en la base de datos.
